# Imports

In [2]:
using LinearAlgebra
using SparseArrays
using Distributions
using PyCall
using Statistics
using BenchmarkTools

@pyimport numpy 

# GMRES

In [110]:
function gmres(A, B, X0, max_iter)
    r = B - A * X0
    x = X0
    q = Array{Any}(zeros(max_iter)) #array any, gdyż przechowuje i wektory i pojedyncze liczby 
    q[1] = r/norm(r)
    h = zeros((max_iter + 1, max_iter))
    
    for m=1:max_iter
        y = A * q[m]
        
        for j=1:m-1
            h[j, m] = (q[j]' * y)[1]
            y = y - h[j, m] * q[j] 
        end
        
        h[m + 1, m] = norm(y)
        
        if (h[m + 1, m] != 0) && (m != max_iter)
            q[m + 1] = y / h[m + 1, m]
        end

        b = zeros(max_iter + 1)
        b[1] = norm(r)
        result = (h \ b)

        if m > max_iter - 2
            z = reduce(hcat, q[:])
        else
            z = q
        end
        
        x = numpy.dot(z, result)+X0
   
    end
    
    return x
end

gmres (generic function with 2 methods)

# Restarted GMRES

In [109]:
function restarted_gmres(A, B, X0, max_iter, res)
    new_x0 = gmres(A, B, X0, res, accuracy)
    for i=2*res:res:max_iter
        new_x0 = gmres(A, B, new_x0, res, accuracy)
    end
    return new_x0
end

restarted_gmres (generic function with 2 methods)

# BICGSTAB

In [114]:
function bicgstab(A, B, init_guess, iterations, accuracy)
    x_prev = init_guess
    r_prev = B - A*x_prev
    r0_arb = r_prev
    rho_prev = alpha = omega_prev = 1.0
    v_prev = p_prev = rho = 0.0
    result = NaN
    for iter = 1:iterations
        rho = dot(r0_arb, r_prev)
        beta = (rho/rho_prev) * (alpha/omega_prev)
        p = r_prev .+ (beta* (p_prev - omega_prev * v_prev))
        v = A * p
        alpha = rho'/(dot(r0_arb, v))
        h = x_prev + alpha*p
        s = r_prev - alpha*v
        if(norm(s) < accuracy)
            result = h
            break
        end
        t = A * s
        omega = dot(t, s) / dot(t, t)
        x = h + omega*s
        r = s - omega*t
        if(norm(r) < accuracy)
            result = x
            break
        end
        r_prev = r
        x_prev = x
        p_prev = p
        v_prev = v
        omega_prev = omega
        rho_prev = rho
    end
    return result
end

bicgstab (generic function with 1 method)

In [6]:
#generacja macierzy A oraz wektorów B i X0
A = Symmetric(sprand(40,40,0.5))
B = rand(Uniform(0,1), 40, 1)
X0 = rand(Uniform(0,1), 40, 1)

40×1 Matrix{Float64}:
 0.2867310900467752
 0.9507270393987897
 0.9171843288642247
 0.32064561268506453
 0.8802631294087755
 0.20746947907597235
 0.869464060184971
 0.853819347919009
 0.84276397307029
 0.23783082191392513
 0.5013404252325211
 0.026314890278079872
 0.3255691059459662
 ⋮
 0.74562344782902
 0.6456796946324386
 0.990812385158695
 0.6578170096152036
 0.22477399524783648
 0.7160939076836463
 0.9614736632872207
 0.9039640228687327
 0.7579941751238493
 0.723459833197156
 0.7975745064661071
 0.3867196598087368

In [116]:
#gmres
max_iter = 10000
res = 150

#bicgstab
accuracy = 1e-5

1.0e-5

In [111]:
x3 = A\B

40×1 Matrix{Float64}:
 -0.7132521694867119
 -0.29341235380365427
 -1.4559211215255188
  0.8250530418009752
  0.15182282649290707
  0.01251409996710051
 -0.4119954864051944
 -0.21002272278729844
  0.23481792633304643
  0.6135130720598987
 -0.5899350747706007
  0.23655938518800915
 -0.4438110862898522
  ⋮
  0.8616213625186604
  0.926974464997958
 -0.994877712460921
 -0.0058144654187628885
  0.39200405277626366
 -0.22705959497074127
  0.4529220622108363
 -0.44375442175462404
  0.7000804830864354
  0.46829996363600596
 -0.884254299228344
 -0.06112872415957583

In [117]:
x2 = restarted_gmres(A, B, X0, max_iter, res)

40×1 Matrix{Float64}:
 -0.7132521694867114
 -0.29341235380365516
 -1.4559211215255206
  0.8250530418009766
  0.15182282649290776
  0.012514099967100876
 -0.41199548640519457
 -0.21002272278729822
  0.23481792633304807
  0.6135130720598999
 -0.5899350747706016
  0.23655938518800967
 -0.44381108628985416
  ⋮
  0.8616213625186612
  0.9269744649979579
 -0.9948777124609222
 -0.005814465418763936
  0.39200405277626377
 -0.22705959497074232
  0.4529220622108376
 -0.44375442175462465
  0.7000804830864354
  0.46829996363600485
 -0.8842542992283449
 -0.06112872415957596

In [115]:
x1 = bicgstab(A, B, X0, max_iter, accuracy)

40×1 Matrix{Float64}:
 -0.713267076397744
 -0.2934155404280447
 -1.4559238616997918
  0.8250262641415805
  0.1518108357513359
  0.012507268553578739
 -0.41199844386292683
 -0.21001660580002213
  0.23478521666361202
  0.613514796071716
 -0.5899149002490177
  0.23655416142129063
 -0.44378284327143924
  ⋮
  0.8616100719034967
  0.9270044560257114
 -0.9948603096987096
 -0.005791593020731345
  0.3920190109851153
 -0.22704873764452269
  0.4528924669206906
 -0.443734953662353
  0.7000937595522012
  0.4683291111775996
 -0.8842639585164662
 -0.06113117686810103

In [167]:
#uruchomienie bazowego gmres
@btime x1 = gmres(A, B, X0, max_iter)

  3.079 s (880164 allocations: 551.34 MiB)


40×1 Matrix{Float64}:
 -0.23204846819451797
 -8.308101367080049
 -2.567115471194662
  1.4508256369342507
  0.4831340884389743
 -4.164164762719872
  3.8632384487738483
  2.6840618379650496
 -2.8874337230338574
 -1.1157877064153898
  0.5034822825095557
  0.37794828944424896
 -1.599963048148649
  ⋮
  4.50823491731588
 -3.9779821139120557
  2.2049374616721344
  1.5586441408086704
 -0.6193247677578106
  4.163307412931417
  0.8833702080808207
  4.947285551984006
  0.12427055400593101
  0.010667145736592332
 -1.1105176686001983
  3.3775810351552003

In [168]:
#uruchomienie restarted gmres
@btime x2 = restarted_gmres(A, B, X0, max_iter*10, res)

  3.036 s (3088175 allocations: 1.11 GiB)


40×1 Matrix{Float64}:
 -0.28540462757055657
 -8.102495879776638
 -2.453003343224105
  1.419308479527588
  0.5084585608950501
 -4.033054599013598
  3.725406033193327
  2.5557361944363515
 -2.881989628385638
 -1.0559358265819117
  0.5081395805858652
  0.3210887879926558
 -1.5530126375830968
  ⋮
  4.380354581793889
 -3.8765806226586905
  2.221376425361447
  1.5086395644035393
 -0.5651573373182394
  3.995037434937755
  0.8643909365585933
  4.834778576252543
  0.1481663967369528
 -0.006967589132115174
 -1.1487776544928883
  3.2423494685363874

In [169]:
#julia linear system solver
x3 = A\B

40×1 Matrix{Float64}:
 -0.25532278236600947
 -8.198219122561914
 -2.5113421107964546
  1.434249642264954
  0.49184416738822717
 -4.096740594700578
  3.793049525608861
  2.6261453449783696
 -2.8776861685827737
 -1.0885906250514517
  0.500397443577207
  0.35423012899185974
 -1.5753573211032137
  ⋮
  4.444602106411869
 -3.9234404956431406
  2.202711403072487
  1.5336807858085424
 -0.5936964952275787
  4.080017202018842
  0.8762546022177471
  4.884134748666695
  0.13406529528867334
  0.0020413069147766872
 -1.1244080195620627
  3.311790010828837

In [170]:
#absolute error
abs_res = broadcast(abs, x2-x3)
abs_normal = broadcast(abs, x1-x3)

mean_abs_res = mean(abs_res)
mean_abs_normal = mean(abs_normal)

println("restarted:")
println(mean_abs_res)
println("base:")
println(mean_abs_normal)

restarted:
0.0034461255578947885
base:
2.4262008063652113e-6


In [171]:
#relative error (percentage)
rel_res = abs_res./x3 * 100
rel_normal = abs_normal./x3 * 100

mean_rel_res = abs(mean(rel_res))
mean_rel_normal = abs(mean(rel_normal))

println("restarted:")
println(mean_rel_res)
println("base:")
println(mean_rel_normal)

restarted:
1.1366593399405238
base:
0.000813883139218001
